# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [ ]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

total= 0
for sqrt_value in generator:
    print(sqrt_value)
    total = total+sqrt_value

print(total)

1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989
32.854555867161245


In [ ]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 13
generator = square_root_generator(limit)

total= 0
for sqrt_value in generator:
    print(sqrt_value)



1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [ ]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


In [ ]:
%%capture
!pip install dlt[duckdb] # Install dlt with all the necessary DuckDB dependencies
!pip install dlt

In [ ]:
import dlt

# define the connection to load to.
# We now use duckdb, but you can switch to Bigquery later
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')

# we can load any generator to a table at the pipeline destnation as follows:
info = generators_pipeline.run(people_1(),	table_name="people_1",	write_disposition="replace")
info_people2 = generators_pipeline.run(people_2(),	table_name="people_1",	write_disposition="append")


In [ ]:
#print(info)

# show outcome

import duckdb

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

people_1_result = conn.sql("SELECT * FROM people_1").df()
display(people_1_result)




Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people_1            │
│ people_2            │
└─────────────────────┘

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708030013.629941,o/Rv2PRKDMVM2A,None
1,2,Person_2,27,City_A,1708030013.629941,xQ5tEbbVmAF2nw,None
2,3,Person_3,28,City_A,1708030013.629941,tCQTWTU/DONXnQ,None
3,4,Person_4,29,City_A,1708030013.629941,I1htOUczMuOD1g,None
4,5,Person_5,30,City_A,1708030013.629941,Ew9hRPHLHm5hRw,None
5,3,Person_3,33,City_B,1708030014.254174,sS8frD2tWj8j4g,Job_3
6,4,Person_4,34,City_B,1708030014.254174,01VjEpIQ9Kx8tw,Job_4
7,5,Person_5,35,City_B,1708030014.254174,JNTy/sHvCkQlHA,Job_5
8,6,Person_6,36,City_B,1708030014.254174,OTMcnJewTes7hg,Job_6
9,7,Person_7,37,City_B,1708030014.254174,0vHbnnlreBYyhg,Job_7


In [ ]:
sum_of_all_ages  = conn.sql("SELECT SUM(age) FROM people_1").df()
display(f'sum_of_all_ages:{sum_of_all_ages}')

'sum_of_all_ages:   sum(age)\n0     353.0'

In [ ]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)

#Creating dlt librarys
!pip install dlt[duckdb] # Install dlt with all the necessary DuckDB dependencies
!pip install dlt

import dlt

# define the connection to load to.
# We now use duckdb, but you can switch to Bigquery later
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')

# we can load any generator to a table at the pipeline destnation as follows:
info = generators_pipeline.run(people_1(),	table_name="people_1",	write_disposition="replace")
info_people2 = generators_pipeline.run(people_2(),	table_name="people_1",	write_disposition="merge", primary_key = "id")

#print(info)

# show outcome

import duckdb

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

people_1_result = conn.sql("SELECT * FROM people_1").df()
display(people_1_result)


sum_of_all_ages  = conn.sql("SELECT SUM(age) FROM people_1").df()
display(f'sum_of_all_ages:{sum_of_all_ages}')

# sum_of_all_ages:   sum(age)\n0     353.0

{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}
Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people_1            │
│ people_2            │
└─────────────────────┘

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708030504.3680465,65eGxH0DEMsLPA,None
1,2,Person_2,27,City_A,1708030504.3680465,ivW9Oc4ojPrUAg,None
2,8,Person_8,38,City_B,1708030504.987184,RuRgpuHchFQn2A,Job_8
3,7,Person_7,37,City_B,1708030504.987184,UZJXDs32pIKo8A,Job_7
4,4,Person_4,34,City_B,1708030504.987184,EZl7fi6uzgGbjg,Job_4
5,3,Person_3,33,City_B,1708030504.987184,n+bIdXK1cMcZ6w,Job_3
6,6,Person_6,36,City_B,1708030504.987184,Wl4mSFC3oWsy7g,Job_6
7,5,Person_5,35,City_B,1708030504.987184,fxRX2FMESz6iIQ,Job_5


'sum_of_all_ages:   sum(age)\n0     266.0'

# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [ ]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [ ]:
# to do: homework :)

#Question 1: What is the sum of the outputs of the generator for limit = 5?

def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

total= 0
for sqrt_value in generator:
    print(sqrt_value)
    total = total+sqrt_value

print(total)

#Question 2: What is the 13th number yielded

def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 13
generator = square_root_generator(limit)

total= 0
for sqrt_value in generator:
    print(sqrt_value)

In [ ]:
#Question 3: Append the 2 generators. After correctly appending the data, calculate the sum of all ages of people.
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)

#Creating dlt librarys
!pip install dlt[duckdb] # Install dlt with all the necessary DuckDB dependencies
!pip install dlt

import dlt

# define the connection to load to.
# We now use duckdb, but you can switch to Bigquery later
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')

# we can load any generator to a table at the pipeline destnation as follows:
info = generators_pipeline.run(people_1(),	table_name="people_1",	write_disposition="replace")
info_people2 = generators_pipeline.run(people_2(),	table_name="people_1",	write_disposition="append")

#print(info)

# show outcome

import duckdb

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

people_1_result = conn.sql("SELECT * FROM people_1").df()
display(people_1_result)


sum_of_all_ages  = conn.sql("SELECT SUM(age) FROM people_1").df()
display(f'sum_of_all_ages:{sum_of_all_ages}')

# sum_of_all_ages:   sum(age)\n0     353.0

In [1]:
#Question 4: Merge the 2 generators using the ID column. Calculate the sum of ages of all the people loaded as described above.

def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)

#Creating dlt librarys
!pip install dlt[duckdb] # Install dlt with all the necessary DuckDB dependencies
!pip install dlt

import dlt

# define the connection to load to.
# We now use duckdb, but you can switch to Bigquery later
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')

# we can load any generator to a table at the pipeline destnation as follows:
info = generators_pipeline.run(people_1(),	table_name="people_1",	write_disposition="replace")
info_people2 = generators_pipeline.run(people_2(),	table_name="people_1",	write_disposition="merge", primary_key = "id")

#print(info)

# show outcome

import duckdb

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

people_1_result = conn.sql("SELECT * FROM people_1").df()
display(people_1_result)


sum_of_all_ages  = conn.sql("SELECT SUM(age) FROM people_1").df()
display(f'sum_of_all_ages:{sum_of_all_ages}')

# sum_of_all_ages:   sum(age)\n0     353.0

Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX